---
# Demo Use Case - ADAS

![overview](adas_overview.png)

A data scientist is part of a team developing an advanced driver-assistance system (ADAS).
They continuously collect video, lidar, and other sensor data from their fleet of test vehicles.
The data scientist wants to test a new object detection model on video collected today.

The raw video is now being continuously ingested into SDP. 
As each video frame is ingested, the object detection model is used to detect objects such as cars, buses, and people.
The object detection model runs as a Flink job in SDP. It uses the YOLO (You Only Look Once) object detection model.
The Flink job uses Java bindings for TensorFlow and it uses GPUs to performance high speed deep learning inference.

Now the data scientist wants to view the result of the object detection model in real-time.
This is shown as a sequence of video frames with labeled boxes around detected objects.
The images from all cameras is combined into the same SDP stream.
Each image is tagged with the original camera number.

### About the video feeds

- We simulate up to 8 simultaneous cameras from different vehicles.
- Each video frame is 1242 x 374 pixels, RGB color, JPEG format.
- JPEG image sizes average 50 KB.
- Each camera records at 10 frames per second.
- The total ingest rate is 5 MB/sec.

---

### How to use this notebook
1. [Install dependencies](install_dependencies.ipynb).
2. [adas_download.ipynb](adas_download.ipynb).
3. [adas_transcode_from_png.ipynb](adas_transcode_from_png.ipynb).
4. [Begin ingesting data](adas_ingest.ipynb).
5. Run Flink job object-detector-job.
6. [Run continuous index builder job](adas_index_builder.ipynb).
7. Run video player. Click *Kernel* -> *Restart Kernel and Run All Cells*.

In [ ]:
#### Import dependencies
%load_ext autoreload
%autoreload 2
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import VideoPlayer
import os
imp.reload(video);

### Define Pravega stream parameters

gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
scope = 'examples'
stream = 'object-detector-output-video'

### Initialize connection to Pravega GRPC Gateway

pravega_channel = grpc.insecure_channel(gateway, options=[
        ('grpc.max_receive_message_length', 9*1024*1024),
    ])
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

---
### Video Player

In [ ]:
VideoPlayer(pravega_client=pravega_client, scope=scope, stream=stream).interact()